In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as graph
import seaborn as sns

In [2]:
%%time
df = pd.read_table('/data4/Ant_genomes/invictus-codon.table')
df.head()

CPU times: user 7min 37s, sys: 3min 20s, total: 10min 58s
Wall time: 12min 3s


In [4]:
display(df.head())

,CHROM,POS,REF,ALT,QUAL,FILTER,AF,AC,AN,DP,EFF
0,NW_020456309.1,379,A,C,71.69,PASS,NaN,NaN,40,142,intergenic_region(MODIFIER|||n.379A>C|||||||C)
1,NW_020456309.1,380,A,C,72.36,PASS,NaN,NaN,40,143,intergenic_region(MODIFIER|||n.380A>C|||||||C)
2,NW_020456309.1,381,A,T,70.49,PASS,NaN,NaN,40,141,intergenic_region(MODIFIER|||n.381A>T|||||||T)
3,NW_020456309.1,385,T,A,76.79,PASS,NaN,NaN,40,140,intergenic_region(MODIFIER|||n.385T>A|||||||A)
4,NW_020456309.1,386,C,G,76.04,PASS,NaN,NaN,40,139,intergenic_region(MODIFIER|||n.386C>G|||||||G)


In [5]:
for allele_column in ['AF', 'AC', 'AN']:
    df[allele_column].fillna(0, inplace=True)

In [6]:
# Calculate folded allele frequency

is_af_less_50 = df['AF'] <= 0.5

df['cal_af'] = np.where(is_af_less_50, df['AC'], df['AN'] - df['AC'])

In [14]:
test = df.sample(n=10000)

In [40]:
df['effect'] = df['EFF'].str.split('(').str.get(0)
df.head()

,CHROM,POS,REF,ALT,QUAL,FILTER,AF,AC,AN,DP,EFF,cal_af,effect
0,NW_020456309.1,379,A,C,71.69,PASS,0.0,0.0,40,142,intergenic_region(MODIFIER|||n.379A>C|||||||C),0.0,intergenic_region
1,NW_020456309.1,380,A,C,72.36,PASS,0.0,0.0,40,143,intergenic_region(MODIFIER|||n.380A>C|||||||C),0.0,intergenic_region
2,NW_020456309.1,381,A,T,70.49,PASS,0.0,0.0,40,141,intergenic_region(MODIFIER|||n.381A>T|||||||T),0.0,intergenic_region
3,NW_020456309.1,385,T,A,76.79,PASS,0.0,0.0,40,140,intergenic_region(MODIFIER|||n.385T>A|||||||A),0.0,intergenic_region
4,NW_020456309.1,386,C,G,76.04,PASS,0.0,0.0,40,139,intergenic_region(MODIFIER|||n.386C>G|||||||G),0.0,intergenic_region


In [41]:
df['genes'] = df['EFF'].str.extract('(XM_[0-9]{9}.[0-9]{1})')
df.head()

,CHROM,POS,REF,ALT,QUAL,FILTER,AF,AC,AN,DP,EFF,cal_af,effect,genes
0,NW_020456309.1,379,A,C,71.69,PASS,0.0,0.0,40,142,intergenic_region(MODIFIER|||n.379A>C|||||||C),0.0,intergenic_region,NaN
1,NW_020456309.1,380,A,C,72.36,PASS,0.0,0.0,40,143,intergenic_region(MODIFIER|||n.380A>C|||||||C),0.0,intergenic_region,NaN
2,NW_020456309.1,381,A,T,70.49,PASS,0.0,0.0,40,141,intergenic_region(MODIFIER|||n.381A>T|||||||T),0.0,intergenic_region,NaN
3,NW_020456309.1,385,T,A,76.79,PASS,0.0,0.0,40,140,intergenic_region(MODIFIER|||n.385T>A|||||||A),0.0,intergenic_region,NaN
4,NW_020456309.1,386,C,G,76.04,PASS,0.0,0.0,40,139,intergenic_region(MODIFIER|||n.386C>G|||||||G),0.0,intergenic_region,NaN


In [66]:
df.drop(columns=['CHROM', 'POS', 'FILTER', 'REF', 'ALT', 'DP', 'EFF'], 
           inplace=True)
display(df.head())

,QUAL,AF,AC,AN,cal_af,effect,genes
0,71.69,0.0,0.0,40,0.0,intergenic_region,NaN
1,72.36,0.0,0.0,40,0.0,intergenic_region,NaN
2,70.49,0.0,0.0,40,0.0,intergenic_region,NaN
3,76.79,0.0,0.0,40,0.0,intergenic_region,NaN
4,76.04,0.0,0.0,40,0.0,intergenic_region,NaN


In [68]:
df['genes'].unique()

array([nan, 'XM_011162030.2', 'XM_011162031.2', ..., 'XM_011175916.2',
       'XM_026141870.1', 'XM_026141872.1'], dtype=object)

In [69]:
df.to_csv('/data4/Ant_genomes/all_ant_invictus_simple.csv', index=False)

In [50]:
# synonymous_snps = df[df['effect'].str.contains('synonymous_variant', na=False)]
synonymous_snps['codons'] = synonymous_snps['EFF'].str.split('(').str.get(1).str.split('|').str.get(2)
synonymous_snps['OG-codon'] = synonymous_snps['codons'].str.split('/').str.get(0)
synonymous_snps['new-codon'] = synonymous_snps['codons'].str.split('/').str.get(1)

/home/arshad/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/arshad/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/arshad/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [51]:
print(len(synonymous_snps))
display(synonymous_snps.head())

2380382


,CHROM,POS,REF,ALT,QUAL,FILTER,AF,AC,AN,DP,EFF,cal_af,effect,genes,codons,OG-codon,new-codon
1152,NW_020456309.1,5846,C,T,73.27,PASS,0.0,0.0,40,107,synonymous_variant(LOW|SILENT|ggC/ggT|p.Gly5Gly/c.15C>T|590|LOC105196222|protein_coding|CODING|XM_011162030.2|2|T),0.0,synonymous_variant,XM_011162030.2,ggC/ggT,ggC,ggT
1158,NW_020456309.1,5852,C,T,67.79,PASS,0.0,0.0,40,108,synonymous_variant(LOW|SILENT|ttC/ttT|p.Phe7Phe/c.21C>T|590|LOC105196222|protein_coding|CODING|XM_011162030.2|2|T),0.0,synonymous_variant,XM_011162030.2,ttC/ttT,ttC,ttT
1160,NW_020456309.1,5915,C,G,64.82,PASS,0.0,0.0,40,105,synonymous_variant(LOW|SILENT|gcC/gcG|p.Ala28Ala/c.84C>G|590|LOC105196222|protein_coding|CODING|XM_011162030.2|2|G),0.0,synonymous_variant,XM_011162030.2,gcC/gcG,gcC,gcG
1162,NW_020456309.1,6071,G,A,71.56,PASS,0.0,0.0,40,112,synonymous_variant(LOW|SILENT|gcG/gcA|p.Ala80Ala/c.240G>A|590|LOC105196222|protein_coding|CODING|XM_011162030.2|2|A),0.0,synonymous_variant,XM_011162030.2,gcG/gcA,gcG,gcA
1166,NW_020456309.1,6075,C,T,73.44,PASS,0.0,0.0,40,113,synonymous_variant(LOW|SILENT|Ctg/Ttg|p.Leu82Leu/c.244C>T|590|LOC105196222|protein_coding|CODING|XM_011162030.2|2|T),0.0,synonymous_variant,XM_011162030.2,Ctg/Ttg,Ctg,Ttg


In [59]:
synon = synonymous_snps.copy()

In [62]:
synon.drop(columns=['CHROM', 'POS', 'FILTER', 'REF', 'ALT', 'DP', 'EFF'], 
           inplace=True)

In [64]:
# synon.to_csv('/data4/Ant_genomes/synonymous_ant_invictus_simple.csv', index=False)